In [11]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [3]:
api_key = "pcsk_79H3n5_RS9wRGp3GA3BzPPp7neaxTszFmr1hizZLfP4JqPftYQQ9pTwLaWE3NbPHLJvuH"

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
#initialize pinecone client

pc = Pinecone(api_key = api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384, #dimension of dense vector
        metric = 'dotproduct', ##sparse values supported only for dot product
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )

In [13]:
index = pc.Index(index_name)
index

/home/akhil/Documents/Langchain-Chatbot/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
## vector embedding, sparse matrix
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
from pinecone_text.sparse import BM25Encoder #TF-IDF

bm25_encoder = BM25Encoder().default()

bm25_encoder

[nltk_data] Downloading package punkt_tab to /home/akhil/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/akhil/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
sentences = [
    "In 2023, I visited Paris",
        "In 2022, I visited New York"
        "In 2021, I visited New Orleans"
]

#tf-idf values on sentences
bm25_encoder.fit(sentences)

#store values in a json file
bm25_encoder.dump("bm25_values.json")



100%|██████████| 2/2 [00:00<00:00, 95.09it/s]


In [23]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder = bm25_encoder, index = index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x77e740efbf10>, index=<pinecone.db_data.index.Index object at 0x77e7429bac20>)

In [24]:
retriever.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited New York"
        "In 2021, I visited New Orleans"
    ]
)
retriever

  0%|          | 0/1 [00:00<?, ?it/s]/home/akhil/Documents/Langchain-Chatbot/langchain_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x77e740efbf10>, index=<pinecone.db_data.index.Index object at 0x77e7429bac20>)

In [25]:
retriever.invoke("What city did I visit first?")

/home/akhil/Documents/Langchain-Chatbot/langchain_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(metadata={'score': 0.25192681}, page_content='In 2022, I visited New YorkIn 2021, I visited New Orleans'),
 Document(metadata={'score': 0.236932307}, page_content='In 2023, I visited Paris')]